## Common preparations

### Importing necessary libs.

In [1]:
import pandas as pd
import os

### Defining paths to data files.

In [2]:
base_dir = os.path.join('data', 'bookmaker')
log_filename = os.path.join(base_dir, 'log.csv')
users_filename = os.path.join(base_dir, 'users.csv')

### Loading log and users data from CSV files into corresponding Pandas DataFrames and merging them.

In [3]:
# Reading log CSV file into Pandas DataFrame
log = pd.read_csv(log_filename, header=None)

# Adding column names
log.columns = ['user_id','time','bet','win']

# Clearing data in time column
def clear_time(time):
    if type(time) == str:
        return time[1:]
    return time

log.time = log.time.apply(clear_time)

# Transforming data in time column to datetime format
log.time = pd.to_datetime(log.time)

# Clearing user ids
# def clear_user_id(user_id):
#     if type(user_id) == str:
#         idx = user_id.find('user')
#         return user_id[idx:]
#     return user_id

# log.user_id = log.user_id.apply(clear_user_id)

# Clearing log from '#error' entries  
log = log[log.user_id != '#error']  
log.user_id = log.user_id.str.split(' - ').apply(lambda x: x[1])

# Checking log DataFrame
display(log.head())
display(log.info())

,user_id,time,bet,win
0,user_919,2019-01-01 14:06:51,NaN,NaN
1,user_973,2019-01-01 14:51:16,NaN,NaN
2,user_903,2019-01-01 16:31:16,NaN,NaN
3,user_954,2019-01-01 17:17:51,NaN,NaN
4,user_954,2019-01-01 21:31:18,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
Int64Index: 985 entries, 0 to 995
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   user_id  985 non-null    object        
 1   time     985 non-null    datetime64[ns]
 2   bet      472 non-null    float64       
 3   win      133 non-null    float64       
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 38.5+ KB


None

In [4]:
# Reading users CSV file into Pandas DataFrame
us = pd.read_csv(users_filename, header=None, encoding='KOI8-R', skiprows=1, sep='\t')

# Adding column names
us.columns = ['user_id','email','geo']

# Transforming user ids into lowercase
us.user_id = us.user_id.apply(lambda x: x.lower())

# Checking us DataFrame
display(us.head())
display(us.info())

,user_id,email,geo
0,user_943,Accumanst@gmail.com,Ижевск
1,user_908,Advismowr@mail.ru,Ижевск
2,user_962,Anachso@ukr.net,Краснодар
3,user_973,Antecia@inbox.ru,Пермь
4,user_902,Balliaryva@ukr.net,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  100 non-null    object
 1   email    99 non-null     object
 2   geo      97 non-null     object
dtypes: object(3)
memory usage: 2.5+ KB


None

In [5]:
# Merging log with us by user_id
log_us = pd.merge(log, us, on='user_id')

In [6]:
# Merged DataFrame examining
display(log_us.head())
display(log_us.info())

,user_id,time,bet,win,email,geo
0,user_919,2019-01-01 14:06:51,NaN,NaN,Chikkaverle@icloud.com,Хабаровск
1,user_919,2019-01-30 10:06:00,NaN,NaN,Chikkaverle@icloud.com,Хабаровск
2,user_919,2019-02-05 14:33:44,NaN,NaN,Chikkaverle@icloud.com,Хабаровск
3,user_919,2019-02-14 11:38:05,NaN,NaN,Chikkaverle@icloud.com,Хабаровск
4,user_919,2019-03-02 04:23:36,300.0,NaN,Chikkaverle@icloud.com,Хабаровск


<class 'pandas.core.frame.DataFrame'>
Int64Index: 975 entries, 0 to 974
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   user_id  975 non-null    object        
 1   time     975 non-null    datetime64[ns]
 2   bet      470 non-null    float64       
 3   win      132 non-null    float64       
 4   email    967 non-null    object        
 5   geo      950 non-null    object        
dtypes: datetime64[ns](1), float64(2), object(3)
memory usage: 53.3+ KB


None

In [7]:
log_us['net'] = log_us['win'] - log_us['bet']

## Tasks.

### Task 1.
Используйте датасет, который получился в результате всех преобразований выше (в том числе, заполнение пропусков). Ответ запишите в поле ниже в виде целого числа (отбросьте дробную часть).
Посчитайте медиану баланса по каждому пользователю. Для этого сгруппируйте по пользователям, возьмите признак net, просуммируйте по каждому пользователю и получите медиану.

In [8]:
log_us.groupby('user_id')['net'].sum().median()

5739.5

### Solution:

### Conclusion: 975 is the right answer!!!

## Using groupby

В данном случае мы группируем данные по признаку user_id.

После этого мы в каждой группе выбираем признак win.

Затем мы берем медиану каждой группы по признаку win и на выходе получаем таблицу, в которой индексом является признак user_id. В этой таблице единственный столбец - медиана по каждой группе (то есть по каждому пользователю).

Наконец, последний вызов median() дает нам медиану по предыдущему столбцу, то есть возвращает одно число.

In [9]:
log_us.groupby('user_id').net.sum().median()

5739.5